## Visualisation de données

inspiration : https://www.kaggle.com/code/datatattle/covid-19-tweets-eda-viz

In [ ]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import defaultdict,Counter
import string
import seaborn as sns
from random import randint

**Création des dataframes d'entrainement et de test**

In [ ]:
train = pd.read_csv("Corona_NLP_train.csv", encoding = 'latin1')
test = pd.read_csv("Corona_NLP_test.csv", encoding = 'latin1')

train['OriginalTweet']=train['OriginalTweet'].astype(str)
train['Sentiment']=train['Sentiment'].astype(str)

test['OriginalTweet']=test['OriginalTweet'].astype(str)
test['Sentiment']=test['Sentiment'].astype(str)

**Affichage des 5 premières ligne du dataframe d'entrainement**

In [ ]:
train.head()

**Affichage des 5 premières ligne du dataframe de test**

In [ ]:
test.head()

**Affichage du nombre et du pourcentage de données manquantes du dataframe d'entrainement**

In [ ]:
null= train.isnull().sum().sort_values(ascending=False)
total =train.shape[0]
percent_missing= (train.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data)

**Affichage du nombre et du pourcentage de données manquantes du dataframe de test**

In [ ]:
null= test.isnull().sum().sort_values(ascending=False)
total =test.shape[0]
percent_missing= (test.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data)

**Affichage du nombre total de tweets dans les donnees d'entrainement ainsi que du nombre d'utilisateurs uniques dans le dataframe d'entrainement**

In [ ]:
print('Total tweets in this data: {}'.format(train.shape[0]))
print('Total Unique Users in this data: {}'.format(train['UserName'].nunique()))

**Affichage du nombre total de tweets dans les donnees d'entrainement ainsi que du nombre d'utilisateurs uniques dans le dataframe de test**

In [ ]:
print('Total tweets in this data: {}'.format(test.shape[0]))
print('Total Unique Users in this data: {}'.format(test['UserName'].nunique()))

**Affichage de la liste des differentes classes**

In [ ]:
print(train.Sentiment.unique())

**Conversion de 5 à 3 classes**

In [ ]:
train['text'] = train.OriginalTweet
train["text"] = train["text"].astype(str)

test['text'] = test.OriginalTweet
test["text"] = test["text"].astype(str)

def classes_def(x):
    if x ==  "Extremely Positive":
        return "positive"
    elif x == "Extremely Negative":
        return "negative"
    elif x == "Negative":
        return "negative"
    elif x ==  "Positive":
        return "positive"
    else:
        return "neutral"
    
train['sentiment']=train['Sentiment'].apply(lambda x:classes_def(x))
test['sentiment']=test['Sentiment'].apply(lambda x:classes_def(x))

**Affichage du nombre total de donnees pour chaque classe pour les donnees d'entrainement et de test**

In [ ]:
fig = make_subplots(1, 2, subplot_titles = ('Train set','Test set'))
fig.update_layout(title_text = "Number of data for each sentiment")
x = train.sentiment.value_counts()
fig.add_trace(go.Bar(x = x.index, y = x.values, marker_color = ['#17C37B','#F92969','#FACA0C'], name = 'train'),row = 1,col = 1)
x=test.sentiment.value_counts()
fig.add_trace(go.Bar(x = x.index, y = x.values, marker_color = ['#17C37B','#F92969','#FACA0C'], name = 'test'),row = 1,col = 2)

**Affichage du nombre total de caracteres par tweet pour chaque classe pour les donnees d'entrainement**

In [ ]:
fig=make_subplots(1,3)
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(15,5))

tweet_len=train[train['sentiment']=="positive"]['text'].str.len()
ax1.hist(tweet_len,color='#17C37B')
ax1.set_title('Positive Sentiments')

tweet_len=train[train['sentiment']=="negative"]['text'].str.len()
ax2.hist(tweet_len,color='#F92969')
ax2.set_title('Negative Sentiments')

tweet_len=train[train['sentiment']=="neutral"]['text'].str.len()
ax3.hist(tweet_len,color='#FACA0C')
ax3.set_title('Neutral Sentiments')

fig.suptitle('Characters in tweets for each sentiment')
plt.savefig("out/characters_in_tweets_for_each_sentiment.png")
plt.show()

**Affichage du nombre total de mots par tweet pour chaque classe pour les donnees d'entrainement**

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(15,5))

tweet_len=train[train['sentiment']=="positive"]['text'].str.split().map(lambda x: len(x))
ax1.hist(tweet_len,color='#17C37B')
ax1.set_title('Positive Sentiments')


tweet_len=train[train['sentiment']=="negative"]['text'].str.split().map(lambda x: len(x))
ax2.hist(tweet_len,color='#F92969')
ax2.set_title('Negative Sentiments')

tweet_len=train[train['sentiment']=="neutral"]['text'].str.split().map(lambda x: len(x))
ax3.hist(tweet_len,color='#FACA0C')
ax3.set_title('Neutral Sentiments')

fig.suptitle('Words in a tweet')
plt.savefig("out/words_in_tweets_for_each_sentiment.png")
plt.show()

**Affichage de la longueur moyenne des mots dans chaque tweet pour chaque classe pour les donnees d'entrainement**

In [ ]:
fig,(ax1,ax2, ax3)=plt.subplots(1,3,figsize=(15,5))

word=train[train['sentiment']=="positive"]['text'].str.split().apply(lambda x : [len(i) for i in x])
sns.histplot(word.map(lambda x: np.mean(x)),ax=ax1,color='#17C37B')
ax1.set_title('Positive')


word=train[train['sentiment']=="negative"]['text'].str.split().apply(lambda x : [len(i) for i in x])
sns.histplot(word.map(lambda x: np.mean(x)),ax=ax2,color='#F92969')
ax2.set_title('Negative')

word=train[train['sentiment']=="neutral"]['text'].str.split().apply(lambda x : [len(i) for i in x])
sns.histplot(word.map(lambda x: np.mean(x)),ax=ax3,color='#FACA0C')
ax3.set_title('Neutral')

fig.suptitle('Average word length in each tweet for each class')
fig.savefig('out/Average word length in each tweet for each class.png')

In [ ]:
def create_corpus(target):
    corpus=[]
    
    for x in train[train['sentiment']==target]['text'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

**Affichage du nombre total d'utilisation de la ponctuation pour tous les tweets pour la classe "Positive" pour les donnees d'entrainement**

In [ ]:
plt.figure(figsize=(10,5))
corpus=create_corpus("positive")

dic=defaultdict(int)

special = string.punctuation
for i in (corpus):
    if i in special:
        dic[i]+=1

x,y=zip(*dic.items())     
plt.title("Total number of uses of punctuation for all tweets for positive sentiment")
plt.bar(x,y,color='#17C37B')
plt.savefig("out/number_punctuation_positive.png")
plt.show()

**Affichage du nombre total d'utilisation de la ponctuation pour tous les tweets pour la classe "Negative" pour les donnees d'entrainement**

In [ ]:
plt.figure(figsize=(10,5))
corpus=create_corpus("negative")

dic=defaultdict(int)
special = string.punctuation
for i in (corpus):
    if i in special:
        dic[i]+=1
        
x,y=zip(*dic.items())
plt.title("Total number of uses of punctuation for all tweets for negative sentiment")
plt.bar(x,y,color='#F92969')
plt.savefig("out/number_punctuation_negative.png")
plt.show()

**Affichage du nombre total d'utilisation de la ponctuation pour tous les tweets pour la classe "Neutral" pour les donnees d'entrainement**

In [ ]:
plt.figure(figsize=(10,5))
corpus=create_corpus("neutral")

dic=defaultdict(int)
import string
special = string.punctuation
for i in (corpus):
    if i in special:
        dic[i]+=1
        
x,y=zip(*dic.items())
plt.title("Total number of uses of punctuation for all tweets for neutral sentiment")
plt.bar(x,y,color='#FACA0C')
plt.savefig("out/number_punctuation_neutral.png")
plt.show()

**Affichage des mots les plus frequents pour chaque dataframe**

In [ ]:
def get_frequencies(words):
    frequencies = {}
    for word in words.split(' '):
        if word in frequencies:
            frequencies[word] += 1
        else:
            frequencies[word] = 1
    return {key: value for key, value in sorted(frequencies.items(),key=lambda item: item[1],reverse=True)}

In [ ]:
def word_cloud(frequencies, title=''):
    wordcloud = WordCloud(width=1000, height=800, background_color='white').generate_from_frequencies(frequencies)
    plt.figure(figsize=(14, 8))
    plt.axis('off')
    plt.imshow(wordcloud)
    plt.title(title, fontsize=30, fontweight='bold')
    plt.savefig("out/"+title+".png")
    plt.show()

**Affichage des mots les plus frequents pour le dataframe d'entrainement**

In [ ]:
index = len(train['OriginalTweet'])-1
words = train['OriginalTweet'][index]
word_cloud(get_frequencies(words), "Most frequent word in the dataframe train")

**Affichage des mots les plus frequents pour le dataframe de test**

In [ ]:
index = len(test['OriginalTweet'])-1
words = test['OriginalTweet'][index]
word_cloud(get_frequencies(words), "Most frequent word in the dataframe test")